In [1]:
import pyspark

In [2]:
import numpy as np

In [62]:
from pyspark.mllib.linalg.distributed import CoordinateMatrix
from pyspark.mllib.linalg.distributed import MatrixEntry
from pyspark.sql.types import *
from operator import add

In [4]:
minimal_fields = [ 
          StructField("author", StringType(), True),
          StructField("score", LongType(), True),
          StructField("controversiality", LongType(), True),
          StructField("subreddit", StringType(), True)]

In [5]:
sj = sqlContext.read.json("s3a://insight-ohoidn/sample3.json", StructType(minimal_fields))

In [5]:
sj.printSchema()

root
 |-- author: string (nullable = true)
 |-- score: long (nullable = true)
 |-- subreddit: string (nullable = true)



In [115]:
sj.toPandas()

,author,score,subreddit
0,stunt_penguin,1,Filmmakers
1,[deleted],2,Addons4Kodi
2,69ing,5,NotTimAndEric
3,ArchadianJudge,2,Saber
4,sglville,2,The_Donald
5,NEWORLDODOR,3,JordanPeterson
6,zachwad22,1,AskReddit
7,lurker4lyfe6969,2,Military
8,SuburbanStoner,2,trashy
9,ensanguine,18,KitchenConfidential


In [6]:
sj.registerTempTable('test')

In [52]:
sqlContext.sql('select author from test').toPandas()

,author
0,stunt_penguin
1,[deleted]
2,69ing
3,ArchadianJudge
4,sglville
5,NEWORLDODOR
6,zachwad22
7,lurker4lyfe6969
8,SuburbanStoner
9,ensanguine


In [7]:
occurrences = sqlContext.sql("""
select *, dense_rank() over (order by subreddit desc) as rid 
from  (SELECT subreddit, author, sum(score) as tally,\
    sum(abs(score)) as activity, dense_rank() over (order by author desc) as uid
from test
group by subreddit, author)
where tally!=0
""").persist(StorageLevel.MEMORY_AND_DISK_SER)
occurrences.registerTempTable('occurrences')

In [75]:
def df_most_active_subreddits(k = 1000):
    most_active = sqlContext.sql("""
    select * from
        (select *, dense_rank() over (order by activity desc) as ordered_id
        from (select rid, subreddit, sum(activity) as activity
            from occurrences
            group by rid, subreddit))
    where ordered_id<=%s
""" % k).persist(StorageLevel.MEMORY_AND_DISK_SER)
    most_active.registerTempTable('most_active')
    return most_active

In [76]:
max_numreddits = 100
act = df_most_active_subreddits(max_numreddits)

In [77]:
act.toPandas()

,rid,subreddit,activity,ordered_id
0,16313,AskReddit,1143263,1
1,2571,politics,205263,2
2,5120,gameofthrones,183469,3
3,441,videos,170466,4
4,15973,BigBrother,156128,5
5,900,todayilearned,109733,6
6,3242,nba,97974,7
7,5109,gaming,95671,8
8,8796,The_Donald,94572,9
9,9396,SquaredCircle,94065,10


In [82]:
def df_valid_users(min_subreddits = 2, max_subreddits = 20):
    """
    filter users by the number of subreddits they've posted in, among the above-defined most active subreddits
    """
    most_active_users = sqlContext.sql("""
    select * from
        (select author, count(subreddit) as count
        from 
            (select * from occurrences
            where subreddit in (select subreddit from most_active))
        group by author
        order by count desc)
    where count>=%d and count<=%d""" % (min_subreddits, max_subreddits)).persist(StorageLevel.MEMORY_AND_DISK_SER)
    most_active_users.registerTempTable('most_active_users')
    return most_active_users

In [83]:
act_users = df_valid_users()

In [84]:
act_users.toPandas()

,author,count
0,Mentioned_Videos,20
1,Agrees_withyou,18
2,JeopardyQBot,16
3,youtubefactsbot,16
4,_youtubot_,15
5,TotesMessenger,14
6,Allenba77,14
7,reelect_rob4d,14
8,RemindMeBot,14
9,LinkFixBot,13


In [85]:
test2 = sqlContext.sql("""
select test.author, test.score, test.controversiality, test.subreddit, most_active.ordered_id as ordered_id
from test
inner join most_active on most_active.subreddit=test.subreddit""")
test2.registerTempTable('test2')

In [86]:
occurrences_pruned = sqlContext.sql("""
select *
from  (SELECT test2.subreddit, author, test2.ordered_id, sum(score) as tally,\
    sum(abs(score)) as activity, dense_rank() over (order by author desc) as uid
    from test2
    where author in (select author from most_active_users)
    group by test2.subreddit, test2.ordered_id, author)
where tally!=0
""").persist(StorageLevel.MEMORY_AND_DISK_SER)
occurrences_pruned.registerTempTable('occurrences_pruned')

In [87]:
occurrences_pruned.toPandas()

,subreddit,author,ordered_id,tally,activity,uid
0,videos,zzzac,4,2,2,1
1,Showerthoughts,zzzac,12,1,1,1
2,The_Donald,zz-zz,9,2,2,2
3,science,zz-zz,68,3,3,2
4,gaming,zywrek,8,1,1,3
5,videos,zywrek,4,1,1,3
6,AskReddit,zywrek,1,15,15,3
7,todayilearned,zyphrinel,6,18,18,4
8,AskReddit,zyphrinel,1,3,3,4
9,videos,zyphrinel,4,19,19,4


In [876]:
occurrences_pruned.rdd.map(lambda rec: rec.rid).max()

101

In [52]:
bare_occurrences = sqlContext.sql("""
select ordered_id as rid, uid, tally
from occurrences_pruned
""").persist(StorageLevel.MEMORY_AND_DISK_SER)

In [886]:
occurrences_pruned.toPandas()

,subreddit,author,tally,activity,uid
0,asoiaf,ztupeztar,643,643,1
1,BlackPeopleTwitter,ztpurcell,190,190,2
2,PoliticalHumor,ztoundas,409,409,3
3,politics,ztoundas,43,43,3
4,niceguys,ztoundas,77,77,3
5,BlackPeopleTwitter,zrsmith3,788,788,4
6,AskReddit,zomskii,489,489,5
7,anime,zikari8,209,209,6
8,gaming,zhivagoo,664,664,7
9,science,zhemao,489,489,8


In [879]:
bare_occurrences.toPandas()

,rid,uid,tally
0,1,32,6
1,1,40,670
2,1,97,13
3,1,105,4
4,1,140,13
5,1,143,-1
6,1,196,-26
7,1,213,3345
8,1,243,1
9,1,261,792


In [658]:
sqlContext.sql("""
select distinct rid, subreddit
from occurrences2
""").toPandas()

,rid,subreddit
0,1,worldnews
1,2,whatisthisthing
2,3,videos
3,4,todayilearned
4,5,therewasanattempt
5,6,teenmom
6,7,technology
7,8,soccer
8,9,skyrim
9,10,science


### Subreddit activity:

In [738]:
df_most_active_subreddits(10).toPandas()

,rid,subreddit,activity,ordered_id
0,16313,AskReddit,1143263,1
1,2571,politics,205263,2
2,5120,gameofthrones,183469,3
3,441,videos,170466,4
4,15973,BigBrother,156128,5
5,900,todayilearned,109733,6
6,3242,nba,97974,7
7,5109,gaming,95671,8
8,8796,The_Donald,94572,9
9,9396,SquaredCircle,94065,10


In [818]:
tf_ij.entries.map(lambda entry: entry.i).distinct().count()

10

In [65]:
gf_i.numCols()

1L

In [67]:
tf_ij.numRows()

101L

In [877]:
act.toPandas()

,rid,subreddit,activity,ordered_id
0,16313,AskReddit,1143263,1
1,2571,politics,205263,2
2,5120,gameofthrones,183469,3
3,441,videos,170466,4
4,15973,BigBrother,156128,5
5,900,todayilearned,109733,6
6,3242,nba,97974,7
7,5109,gaming,95671,8
8,8796,The_Donald,94572,9
9,9396,SquaredCircle,94065,10


In [63]:
# subreddit activity vector
gf_i = CoordinateMatrix(act.rdd.map(lambda row: (row.ordered_id, 0, row.activity)))

# subreddit-activity matrix
tf_ij = CoordinateMatrix(bare_occurrences.rdd.map(tuple))
nusers = tf_ij.numCols()

p_ij = coordinate_matrix_elementwise_vector_division(tf_ij, gf_i)

# TODO figure out nan issues
logp_ij = coordinateMatrixElementwise(p_ij, lambda elt: np.log((abs(elt))/np.log(nusers)))

entropy_i = coordinateMatrixElementwise(coordinate_matrix_sumj(logp_ij), lambda elt: elt + 1) # + 1, but this has to be handled separately to conserve matrix sparsity

In [826]:
def transform_tf_if(val):
    newval = np.log(np.abs(val) + 1)
    if val < 0:
        return -newval
    else:
        return newval

# TODO: maybe save this for later, when noise can be reduced via dimensionality reduction
log_tf_ij = coordinateMatrixElementwise(tf_ij, transform_tf_if)

a_ij = coordinatematrix_multiply_vector_elementwise(log_tf_ij, entropy_i)

In [844]:
def fast_similarity(ratings, epsilon=1e-9):
    # epsilon -> small number for handling dived-by-zero errors
    sim = ratings.dot(ratings.T) + epsilon
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return (sim / norms / norms.T)

In [845]:

def predict_nobias(ratings, similarity, kind='user'):
    if kind == 'user':
        user_bias = ratings.mean(axis=1)
        ratings = (ratings - user_bias[:, np.newaxis]).copy()
        pred = similarity.dot(ratings) / np.array([np.abs(similarity).sum(axis=1)]).T
        pred += user_bias[:, np.newaxis]
    elif kind == 'item':
        item_bias = ratings.mean(axis=0)
        ratings = (ratings - item_bias[np.newaxis, :]).copy()
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
        pred += item_bias[np.newaxis, :]
        
    return pred

In [846]:
from sklearn.metrics import mean_squared_error

def get_mse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

In [843]:
np.ones((2, 2)).dot(np.ones((2, 1)))

array([[ 2.],
       [ 2.]])

In [756]:
bare_occurrences.toPandas()

,rid,uid,tally
0,1,173573,1
1,2,2510,1
2,3,202843,1
3,4,71343,2
4,5,157008,1
5,6,22113,2
6,6,29476,11
7,6,41712,6
8,6,52207,4
9,6,68993,9


In [796]:
a_ij.numCols()

57010L

In [828]:
ndmat = coordinate_matrix_to_ndarr(p_ij)
ndmat/=np.mean(ndmat)

In [835]:
np.max(ndmat)

108065.21478879372

In [337]:
a_ij.entries.count()

465503

In [325]:
tf_ij.numRows()

17045L

In [323]:
entropy_i.numRows()

17045L

In [ ]:
logp_ij = coordinateMatrixElementwise

In [203]:
activity_df.toPandas()

,rid,activity
0,1,1
1,2,1
2,3,1
3,4,2
4,5,1
5,6,295
6,7,32
7,8,9
8,9,4
9,10,3


In [9]:
from operator import add

# TODO: check that zero entries are correctly filtered
def coordinateMatrixMultiply(leftmat, rightmat):
    m = leftmat.entries.map(lambda entry: (entry.j, (entry.i, entry.value)))
    n = rightmat.entries.map(lambda entry: (entry.i, (entry.j, entry.value)))
    product_entries = m.join(n)\
    .map(lambda tup: ((tup[1][0][0], tup[1][1][0]), (tup[1][0][1] * tup[1][1][1])))\
    .reduceByKey(add)\
    .map(lambda record: MatrixEntry(record[0][0], record[0][1], record[1]))
    
    return pyspark.mllib.linalg.distributed.CoordinateMatrix(product_entries)

In [121]:
def coordinateMatrixAdd(leftmat, rightmat, scalar):
    """
    Return leftmat + scalar * rightmat
    """
    m = leftmat.entries.map(lambda entry: ((entry.i, entry.j), entry.value))
    n = rightmat.entries.map(lambda entry: ((entry.i, entry.j), scalar * entry.value))
    matsum = m.fullOuterJoin(n)\
    .map(lambda tup: MatrixEntry(tup[0][0], tup[0][1],
                                 reduce(add, filter(lambda elt: elt is not None, tup[1]))))
    
    #return matsum
    return pyspark.mllib.linalg.distributed.CoordinateMatrix(matsum)

In [56]:
def coordinate_matrix_elementwise_vector_division(mat, vec):
    """
    mat : CoordinateMatrix
    
    mat_{ij} -> mat_{ij}/vec_{i}
    """
    m = mat.entries.map(lambda entry: (entry.i, (entry.j, entry.value)))
    v = vec.entries.map(lambda entry: (entry.i, (entry.j, entry.value)))
    matdiv = m.join(v).map(lambda tup: MatrixEntry(tup[0], tup[1][0][0], float(tup[1][0][1]) / tup[1][1][1]))
    
    return pyspark.mllib.linalg.distributed.CoordinateMatrix(matdiv)

In [260]:
#from operator import multipl
def coordinate_matrix_elementwise_matrix_multiplication(mat1, mat2):
    """
    mat : CoordinateMatrix
    
    return matprod, where matprod_{ij} = mat1_{ij} * mat2_{ij}
    """
    m1 = mat1.entries.map(lambda entry: ((entry.i, entry.j), entry.value))
    m2 = mat2.entries.map(lambda entry: ((entry.i, entry.j), entry.value))
    
    matprod = m1.join(m2).map(lambda tup: MatrixEntry(tup[0][0], tup[0][1], tup[1][0] * tup[1][1]))
    #matprod = m1.join(m2).reduceByKey(lambda tup: MatrixEntry(tup[0][0], tup[0][1], tup[1][0] * tup[1][1]))
    return pyspark.mllib.linalg.distributed.CoordinateMatrix(matprod)

In [60]:
def coordinate_matrix_sumj(mat):
    """
    mat : CoordinateMatrix
    """
    summed_entries = mat.entries.map(lambda entry: (entry.i, entry.value)).reduceByKey(add)\
                          .map(lambda tup: MatrixEntry(tup[0], 0, tup[1]))

    return pyspark.mllib.linalg.distributed.CoordinateMatrix(summed_entries)

In [347]:
def coordinate_matrix_row(mat, i):
    """
    mat : CoordinateMatrix
    
    return the specified row vector
    """
    filtered_entries = mat.entries.filter(lambda entry: entry.i == i).\
        map(lambda entry: MatrixEntry(0, entry.j, entry.value))
    return pyspark.mllib.linalg.distributed.CoordinateMatrix(filtered_entries)

In [355]:
def coordinate_vector_matrix_norm(vec):
    """
    TODO: type checking, confusing name?
    """
    return np.sqrt(vec.entries.map(lambda entry: entry.value**2).sum())

In [360]:
test_vec = np.array([[1,1,0]]).T
test_vec2 = np.array([[1,0,0]]).T

In [356]:
coordinate_vector_matrix_norm(ndarr_to_coord_array(test_vec))

1.4142135623730951

In [358]:
def coordinate_matrix_vector_l2(vec1, vec2):
    """
    Given two vectors of the data type CoordinateMatrix, return L2 norm of vec1/|vec1| - vec2/|vec2|
    """
    norm1, norm2 = map(coordinate_vector_matrix_norm, [vec1, vec2])

    vec1normed = coordinateMatrixScalarMult(vec1, 1./norm1)
    vec2normed = coordinateMatrixScalarMult(vec2, 1./norm2)
    
    diff = coordinateMatrixAdd(vec1normed, vec2normed, -1.)
    return coordinate_vector_matrix_norm(diff)

In [661]:
def sort_row_indices_by_distance(mat, vec):
    """
    Return a list of the row indices of mat sorted by the ascending L2 distance between normalized row vectors and vec/|vec|
    """
    size = mat.numCols()
    row_vectors = mat.entries.map(lambda entry: (entry.i, [(entry.j, entry.value)]))\
        .reduceByKey(add).map(lambda tup: (tup[0], Vectors.sparse(size, tup[1])))
    # TODO replace all 1D CoordinateMatrix instances by local sparse vectors
    compare_vector = vec.entries.map(lambda entry: ('', [(entry.j, entry.value)])).reduceByKey(add)\
        .map(lambda tup: Vectors.sparse(size, tup[1])).collect()[0]
    # TODO check vector normalization
    return row_vectors\
        .map(lambda tup: (tup[1].squared_distance(compare_vector)/(tup[1].norm(2) * compare_vector.norm(2)), tup[0])).sortByKey()\
        .map(lambda tup: tup[1]).collect()

In [553]:
def coordinatematrix_get_row(mat, i):
    return pyspark.mllib.linalg.distributed.CoordinateMatrix(
                    mat.entries.filter(lambda entry: entry.i == i))

In [665]:
sort_row_indices_by_distance(a_ij, coordinatematrix_get_row(a_ij, 37))

[33L,
 3L,
 36L,
 6L,
 39L,
 9L,
 42L,
 12L,
 45L,
 15L,
 48L,
 18L,
 51L,
 21L,
 54L,
 24L,
 57L,
 27L,
 30L,
 1L,
 34L,
 4L,
 37L,
 7L,
 40L,
 10L,
 43L,
 13L,
 46L,
 16L,
 49L,
 19L,
 52L,
 22L,
 55L,
 25L,
 58L,
 28L,
 31L,
 32L,
 2L,
 35L,
 5L,
 38L,
 8L,
 41L,
 11L,
 44L,
 14L,
 47L,
 17L,
 50L,
 20L,
 53L,
 23L,
 56L,
 26L,
 59L,
 29L]

In [ ]:
sort_row_indices_by_distance()

In [484]:
test_array5 = 2 * np.array([[1, 0, 0], [1, 1, 0], [0, 0,  1]])
vec = 2 * np.array([[1, 1, 0.001]])

In [487]:
cm = ndarr_to_coord_array(test_array5)
cv = ndarr_to_coord_array(vec)

In [563]:
td = coordinatematrix_get_row(cmat, 8797)

In [564]:
td.entries.count()

1

In [537]:
sort_row_indices_by_distance(cm, cv)

[1L, 0L, 2L]

In [474]:
from pyspark.mllib.linalg import Vectors, ArrayType

In [532]:
v = Vectors.sparse(3, [(1, 1), (2, 1)])

In [533]:
v.norm(2)

1.4142135623730951

In [492]:
cv.entries.map(lambda entry: (entry.j, entry.value)).collect()

[(0L, 2.0), (1L, 2.0), (2L, 0.002)]

In [493]:
def coordinatematrix_to_sparse_vector(mat):
    """
    mat : CoordinateMatrix
    
    Mat is assumed to have non-zero entries in only the 0th row index
    """
    size = mat.numCols()
    return Vectors.sparse(size, mat.entries.map(lambda entry: (entry.j, entry.value)).collect())

In [495]:
v = coordinatematrix_to_sparse_vector(cv)

In [471]:
mat_j.join(vec_j).

[(0L, ((0L, 2.0), 2.0)),
 (0L, ((1L, 2.0), 2.0)),
 (1L, ((1L, 2.0), 2.0)),
 (2L, (None, 0.002))]

In [466]:
mat_j.fullOuterJoin(vec_j).map(lambda tup: (tup[1][0][0], (tup[1][0][1] - tup[1][1])**2)).collect()

[(1L, 0.0), (0L, 0.0), (1L, 0.0), (2L, 3.992004)]

In [454]:
mat_j.join(vec_j).map(lambda tup: (tup[1][0][0], (tup[1][0][1] - tup[1][1])**2)).reduceByKey(add).collect()

[(0L, 0.0), (1L, 0.0)]

In [453]:
mat_j.join(vec_j).map(lambda tup: (tup[1][0][0], (tup[1][0][1] - tup[1][1])**2)).reduceByKey(add)\
    .map(lambda tup: (tup[1], tup[0])).sortByKey().map(lambda tup: tup[1]).collect()

[0L, 1L]

In [439]:
ndarr_to_coord_array(test_array2).entries.map(lambda entry: (entry.i, [(entry.i, entry.j)]))\
.reduceByKey(add).collect()

[(0L, [(0L, 0L), (0L, 1L), (0L, 2L)]),
 (1L, [(1L, 0L), (1L, 2L), (1L, 3L)]),
 (2L, [(2L, 2L), (2L, 3L)])]

In [361]:
coordinate_matrix_vector_l2(ndarr_to_coord_array(test_vec), ndarr_to_coord_array(test_vec))

0.0

In [371]:
vec1, vec2 = ndarr_to_coord_array(test_vec), ndarr_to_coord_array(test_vec2)

In [372]:
norm1, norm2 = map(coordinate_vector_matrix_norm, [vec1, vec2])

vec1normed = coordinateMatrixScalarMult(vec1, 1./norm1)
vec2normed = coordinateMatrixScalarMult(vec2, 1./norm2)

diff = coordinateMatrixAdd(vec1normed, vec2normed, -1.)

In [373]:
diff.entries.collect()

[MatrixEntry(0, 0, -0.292893218813), MatrixEntry(1, 0, 0.707106781187)]

In [380]:
coordinate_matrix_vector_l2(ndarr_to_coord_array(test_vec), ndarr_to_coord_array(2 * test_vec2))

0.76536686473017945

In [366]:
test_vec2


array([[1],
       [0],
       [0]])

In [379]:
np.linalg.norm(test_vec.T[0] / np.linalg.norm(test_vec) - test_vec2.T[0] / np.linalg.norm(test_vec2))

0.76536686473017945

In [375]:
mat = ndarr_to_coord_array(test_array2)
mat.entries.filter(lambda entry: entry.i == 2).map(lambda entry: collect()

SyntaxError: invalid syntax (<ipython-input-375-dc50cd5dbed2>, line 2)

In [298]:
def coordinatematrix_multiply_vector_elementwise(mat, vec):
    """
    mat : CoordinateMatrix
    vec : CoordinateMatrix
    """
    mat_entries = mat.entries.map(lambda entry: (entry.i, (entry.j, entry.value)))
    vec_entries = vec.entries.map(lambda entry: (entry.i, entry.value))

    prod = vec_entries.join(mat_entries).map(lambda tup: MatrixEntry(tup[0], tup[1][1][0], tup[1][0] * tup[1][1][1]))
    return pyspark.mllib.linalg.distributed.CoordinateMatrix(prod)

In [58]:
def coordinateMatrixElementwise(mat, op):
    """
    elt -> op(elt) for each nonzero element elt of the matrix mat
    """
    new_entries = mat.entries.map(lambda entry: MatrixEntry(entry.i, entry.j, op(entry.value)))
    return pyspark.mllib.linalg.distributed.CoordinateMatrix(new_entries)

In [ ]:
def coordinateMatrixElementwiseMultiplication(mat, scalar):
    """
    return scalar * mat
    """
    new_entries = mat.entries.map(lambda entry: MatrixEntry(entry.i, entry.j, scalar * entry.value))
    return pyspark.mllib.linalg.distributed.CoordinateMatrix(new_entries)

Implement LSA

Test stuff

In [94]:
test_coordmat2 = ndarr_to_coord_array(test_array2)

In [97]:
test_coordmat2.entries.filter(lambda entry: entry.value != 0).collect()

[MatrixEntry(0, 0, 1.0),
 MatrixEntry(0, 1, 2.0),
 MatrixEntry(0, 2, 3.0),
 MatrixEntry(1, 0, 4.0),
 MatrixEntry(1, 2, 6.0),
 MatrixEntry(1, 3, 1.0),
 MatrixEntry(2, 2, 8.0),
 MatrixEntry(2, 3, 2.0)]

In [123]:
def coordinate_matrix_to_ndarr(mat):
    size = mat.entries.count()
    elts = mat.entries.take(size)
    arr = np.zeros((mat.numRows(), mat.numCols()))
    for elt in elts:
        arr[elt.i][elt.j] = elt.value
    return arr

def ndarr_to_coord_array(arr):
    entries = []
    for i in range(len(arr)):
        for j in range(len(arr[0])):
            if arr[i][j] != 0:
                entries.append((i, j, arr[i][j]))
    return CoordinateMatrix(sc.parallelize(entries))

test_array = np.array([[1, 2, 3], [4, 0, 6], [0, 0, 8]], dtype = 'float')
test_array2 = np.array([[1, 2, 3, 0], [4, 0, 6, 1], [0, 0, 8, 2]], dtype = 'float')
test_array1d = np.array([[1, 1, 1, 1]])
test_coordmat = ndarr_to_coord_array(test_array)
test_coordmat2 = ndarr_to_coord_array(test_array2)
test_coordmat2_T = ndarr_to_coord_array(test_array2.T)


def test_multiply():
    assert np.all(coordinate_matrix_to_ndarr(coordinateMatrixMultiply(test_coordmat, test_coordmat)) ==\
        np.dot(test_array, test_array))
    assert np.all(coordinate_matrix_to_ndarr(coordinateMatrixMultiply(test_coordmat2, test_coordmat2_T)) ==\
        np.dot(test_array2, test_array2.T))
    #assert coordinate_matrix_to_ndarr(coordinateMatrixMultiply(
    #    ndarr_to_coord_array(test_array1d), ndarr_to_coord_array(test_array1d.T)))
    
test_array3 = np.array([[ 0.63203118,  0.30233108,  0.40677762,  0.58962667],
       [ 0.98905039,  0.9516414 ,  0.20273982,  0.20800506],
       [ 0.7751541 ,  0.94623161,  0.22601002,  0.40736821]])

def test_add():
    m = coordinate_matrix_to_ndarr(
        coordinateMatrixAdd(ndarr_to_coord_array(test_array3), ndarr_to_coord_array(test_array2), -2))
    m2 = test_array3 - 2 * test_array2
    #return m, m2
    assert np.all(m == m2)

In [101]:
coordinateMatrixAdd(ndarr_to_coord_array(test_array3), ndarr_to_coord_array(test_array2), -2)

In [90]:
coordinate_matrix_to_ndarr(coordinateMatrixMultiply(
        ndarr_to_coord_array(test_array1d), ndarr_to_coord_array(test_array1d.T)))

array([[ 4.]])

In [217]:
def eval_matrix_binop(ndarr1, ndarr2, op):
    cmat = op(ndarr_to_coord_array(ndarr1),
             ndarr_to_coord_array(ndarr2))
    ndmat = coordinate_matrix_to_ndarr(cmat)
    return cmat, ndmat

In [244]:
vec = ndarr_to_coord_array(np.array([[2, 2, 2, 2]]))
mat = ndarr_to_coord_array(np.ones((4, 4)))

m = mat.entries.map(lambda entry: (entry.i, (entry.j, entry.value)))
v = vec.entries.map(lambda entry: (entry.i, (entry.j, entry.value)))
matdiv = m.join(v).map(lambda tup: MatrixEntry(tup[0], tup[1][0][0], float(tup[1][0][1]) / tup[1][1][1]))

In [263]:
eval_matrix_binop(2 * np.ones((4, 4)), 2 * np.ones((4, 4)), coordinate_matrix_elementwise_matrix_multiplication)

(<pyspark.mllib.linalg.distributed.CoordinateMatrix at 0x7fde2b3eba90>,
 array([[ 4.,  4.,  4.,  4.],
        [ 4.,  4.,  4.,  4.],
        [ 4.,  4.,  4.,  4.],
        [ 4.,  4.,  4.,  4.]]))

In [269]:
sumjtest = coordinate_matrix_sumj(ndarr_to_coord_array(np.ones((3, 4))))

In [270]:
coordinate_matrix_to_ndarr(sumjtest)

array([[ 4.],
       [ 4.],
       [ 4.]])

In [296]:
test_array2

array([[ 1.,  2.,  3.,  0.],
       [ 4.,  0.,  6.,  1.],
       [ 0.,  0.,  8.,  2.]])

In [300]:
eval_matrix_binop(test_array2, np.array([[2, 3, 4]]).T, coordinatematrix_multiply_vector_elementwise)

(<pyspark.mllib.linalg.distributed.CoordinateMatrix at 0x7fde20ca84d0>,
 array([[  2.,   4.,   6.,   0.],
        [ 12.,   0.,  18.,   3.],
        [  0.,   0.,  32.,   8.]]))

In [351]:
coordinate_matrix_row(ndarr_to_coord_array(test_array2), 1).entries.collect()

[MatrixEntry(0, 0, 4.0), MatrixEntry(0, 2, 6.0), MatrixEntry(0, 3, 1.0)]

In [125]:
test_add()
test_multiply()

In [78]:
coordinate_matrix_to_ndarr(coordinateMatrixScalarMult(ndarr_to_coord_array(test_array2), 1.5))

array([[  1.5,   3. ,   4.5,   0. ],
       [  6. ,   0. ,   9. ,   1.5],
       [  0. ,   0. ,  12. ,   3. ]])

In [92]:
coordinate_matrix_to_ndarr(coordinateMatrixElementwise(ndarr_to_coord_array(test_array2), lambda n: np.log(n)))

array([[ 0.        ,  0.69314718,  1.09861229,        -inf],
       [ 1.38629436,        -inf,  1.79175947,  0.        ],
       [       -inf,        -inf,  2.07944154,  0.69314718]])

In [93]:
test_array2

array([[ 1.,  2.,  3.,  0.],
       [ 4.,  0.,  6.,  1.],
       [ 0.,  0.,  8.,  2.]])

SVD implementation

In [ ]:
dfi = 

In [80]:
from random import normalvariate


In [85]:
cmat.numCols()

347112L

In [83]:
from random import normalvariate
def random_unit(n):
    vec = np.array([normalvariate(0, 1) for _ in range(n)])
    return vec / np.linalg.norm(vec)

def SVD_1D(A, threshold = 1e-10):
    """
    A : CoordinateMatrix
    """
    n, m = A.numRows(), A.numCols()
    x = random_unit(m)
    # TODO finish this

In [ ]:
import numpy as np
from numpy.linalg import norm

def svd_1d(A, epsilon=1e-10):
    ''' The one-dimensional SVD '''
    n, m = A.shape
    x = randomUnitVector(m)
    lastV = None
    currentV = x
    B = np.dot(A.T, A)
 
    iterations = 0
    while True:
        iterations += 1
        lastV = currentV
        currentV = np.dot(B, lastV)
        currentV = currentV / norm(currentV)
 
        if abs(np.dot(currentV, lastV)) > 1 - epsilon:
            print("converged in {} iterations!".format(iterations))
            return currentV

In [62]:
m2 = coordinateMatrixAdd(cmat, cmat, -2)

In [64]:
m2.entries.take(2)

[MatrixEntry(3, 9, -2.0), MatrixEntry(8, 4, -1.0)]

In [25]:
matsum = coordinateMatrixAdd(ndarr_to_coord_array(test_array3), ndarr_to_coord_array(test_array3), -2)

In [29]:
cmat = CoordinateMatrix(bare_occurrences.rdd.map(tuple))

In [43]:
cmat.entries.take(2)

[MatrixEntry(1, 176355, 1.0), MatrixEntry(2, 2547, 1.0)]

In [74]:
mul.entries.count()

595811

In [41]:
cmat.entries.count()

475392

In [45]:
cmb = cmat.toBlockMatrix()

In [47]:
cmi = cmb.toIndexedRowMatrix()

In [51]:
row = cmi.rows.take(1)

[IndexedRow(3558, (347112,[54015,294324],[1.0,2.0]))]

In [144]:
sqlContext.sql("""
select max(uid)
from occurrences
""").toPandas()

,max(uid)
0,341578


In [57]:
occurrences.toPandas()

,subreddit,author,tally,uid,rid
0,zyramains,TricolorStar,1,176355,1
1,zylzon,youknowitsyaboy,1,2547,2
2,zweiteliga,Saminka,1,206115,3
3,zurich,maxwellmaxen,2,72503,4
4,zsh,__soddit,1,159502,5
5,zootopia,thawed_caveman,2,22475,6
6,zootopia,speisekarte,11,29937,6
7,zootopia,rodrigogirao,6,42380,6
8,zootopia,phobos136,4,53058,6
9,zootopia,midnightopheliac,9,70104,6


In [89]:
# TODO: why doesn't this match rid?
cmat.numRows()

17085L

In [71]:
import numpy as np

In [74]:
np.array(df['tally']).min()

-632

In [69]:
sqlContext.sql("""
select author, subreddit, sum (score)
from test
group by author, subreddit
""").toPandas()

,author,subreddit,sum(score)
0,FormulaXDGame,formula1,31
1,HelluvaDeke,gaming,2
2,WhoKnowsWho2,xboxone,70
3,listentohim,DBZDokkanBattle,1
4,sevenzig,AskOuija,10
5,LumpyPick,leagueoflegends,4
6,volvostupidshit,Philippines,6
7,i_fuck_goats777,AskReddit,2
8,Ineeditunesalot,battlefield_4,11
9,martianlawrence,movies,11


In [78]:
sj.toPandas()

,author,author_cakeday,author_flair_css_class,author_flair_text,body,can_gild,collapsed,collapsed_reason,controversiality,created_utc,distinguished,edited,gilded,id,is_submitter,link_id,parent_id,retrieved_on,score,stickied,
0,stunt_penguin,None,None,None,Wheelchairs make amazing dollys :D,True,False,None,0,1501545600,None,false,0,dkznc8h,False,t3_6qp8sw,t1_dkzbnn3,1503654247,1,False,...
1,[deleted],None,None,None,[removed],True,False,None,0,1501545600,None,false,0,dkznc8i,False,t3_6qse6i,t1_dkzmgc3,1503654247,2,False,...
2,69ing,None,None,None,I used to watch the shit out of these guys,True,False,None,0,1501545600,None,false,0,dkznc8j,False,t3_6qs8i1,t3_6qs8i1,1503654247,5,False,...
3,ArchadianJudge,None,Archadianflair,None,http://www.pixiv.net/member_illust.php?mode=me...,True,False,None,0,1501545600,None,false,0,dkznc8k,True,t3_6qsicx,t3_6qsicx,1503654247,2,False,...
4,sglville,None,None,None,On the other yand you could say it's capitalis...,True,False,None,0,1501545600,None,false,0,dkznc8l,False,t3_6qryxf,t3_6qryxf,1503654247,2,False,...
5,NEWORLDODOR,None,None,None,I'm not arguing that making university free an...,True,False,None,0,1501545600,None,false,0,dkznc8m,True,t3_6qrr1o,t1_dkzn053,1503654247,3,False,...
6,zachwad22,None,None,None,"I like whisper-whistle obsessively, just barel...",True,False,None,0,1501545600,None,false,0,dkznc8n,False,t3_6qoe6s,t3_6qoe6s,1503654247,1,False,...
7,lurker4lyfe6969,None,None,None,Served in the Chair Force. Can confirm,True,False,None,0,1501545600,None,false,0,dkznc8o,False,t3_6qodwi,t3_6qodwi,1503654247,2,False,...
8,SuburbanStoner,None,None,None,"Nah, it's good right here",True,False,None,0,1501545600,None,false,0,dkznc8p,False,t3_6qn002,t1_dkz6ig2,1503654247,2,False,...
9,ensanguine,None,None,None,I still stand by it not belonging on the plate...,True,False,None,0,1501545600,None,false,0,dkznc8q,False,t3_6qscet,t1_dkzn9vn,1503654247,18,False,...


In [27]:
rawDF = sqlContext.read.json("s3n://reddit-comments/2015/RC_2015-05", StructType(fields)).persist(StorageLevel.MEMORY_AND_DISK_SER)

In [28]:
rawDF.registerTempTable('comments')